# Práctica guiada - Join con Pandas

# PARTE I

* Algunas de las operaciones más interesantes con los datos provienen de la combinación de diferentes fuentes de datos. Estas operaciones pueden ser

    1. simples concatenaciones de datos de datasets diferentes
    2. operaciones más similares a un join o merge en una base de datos

* Tanto  las `Series` como `DataFrames` fueron construidos teniendo estas operaciones en mente e incluyen funciones y métodos para realizarlas de forma rápida y simple.

* Veremos dos operaciones: `pd.append` y `pd.concat`

In [48]:
import pandas as pd
import numpy as np
from IPython.display import display

Vamos a crear una función que crea un `DataFrame` para simplificar algunos pasos:

In [49]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


## Concatenación simple con ``pd.concat``

* ``pd.concat()``, permite hacer concatenaciones simples de diferentes `Series`.

* Tiene una sintaxis similar a su análoga en Numpy ``np.concatenate``. Pero contiene algunas opciones más:

```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

In [50]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

También permite concater objetos de mayor dimensionalidad como `DataFrame`:

In [51]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])

display(df1, df2)

pd.concat([df1,df2])

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


* Por defecto, la concatenación se hace en el sentido de las filas del ``DataFrame`` (i.e., ``axis=0``).
* Puede especificarse el eje sobre cual realizar la concatenación:

In [52]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])

display(df3, df4)
        
pd.concat([df3, df4], axis=1)

,A,B
0,A0,B0
1,A1,B1


,C,D
0,C0,D0
1,C1,D1


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


### Índice duplicados

Recordemos que numpy también presenta un método de concatenación.

Una diferencia importante entre  ``np.concatenate`` y ``pd.concat`` es que la concatenación de Pandas preserva los índices, incluso si el resultado implica índices duplicados:

In [53]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # índices duplicados!
display(x,y)

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


In [54]:
pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Si bien `DataFrame` permite índices duplicados es preferible eviatarlos.

#### Verificando la existencia índices duplicados

* Podemos verificar si existen índices solapados en el resultado de ``pd.concat()`` usando una ``verify_integrity`` flag.

* Al setear esto en True, la concatenación arrojará una excepción si existe algún índice duplicado:

In [55]:
# El parámetro verify_integrity evita que concatenemos dataframes con índices iguales.
pd.concat([x, y], verify_integrity=True)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')

In [56]:
# En general, para no recibir este comportamiento frente a un error, podemos generar una excepción para mostrarselo
# al usuario de forma más prolija
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignorando el índice

* En algunos casos el índice no importa u podemos ignorarlo: simplemente usamos nuevamente ``ignore_index``. Al setearlo en True, se genera un nuevo índice.

In [57]:
display(x,y)
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1


,A,B
0,A2,B2
1,A3,B3


,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### El método ``append()``

* Dado que la concatenación de arrays es bastante común, ``Series`` y ``DataFrame``tienen un método ``append`` 

* Por ejemplo, en lugar de llamar ``pd.concat([df1, df2])``, es posible llamar al más sencillo ``df1.append(df2)``:

In [58]:
display(df1,df2)
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [59]:
df1

,A,B
1,A1,B1
2,A2,B2


* **IMPORTANTE:** tener en cuenta que, a diferencia del método `append` de listas, el `append()` de Pandas no modifica el objeto original. Genera un nuevo objeto con los datos combinados.
* Dado que esto implica crear un índice nuevo y un nuevo dataset `append` puede no ser el mejor método si se planea concatenar muchos datasets seguidos.
* En estos casos es mejor usar la función `pd.concat()`.

# PARTE II 

## Tipos de relaciones

Una de las características más valiosas de la librería Pandas es su funcionalidad para realizar joins en memoria y de manera eficiente.

El método merge() permite trabajar con objetos que presentan distintos tipos de relaciones:
 1. Uno a uno
 2. Muchos a uno
 3. Muchos a muchos
 

### 1. Join uno a uno

In [60]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display(df1)
display(df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [61]:
# Vemos que cada empleado tiene un sólo grupo y una sola fecha de contratación.
# Combinamos los dataframes usando pd.merge()
# Notar que la función merge encontró la única columna en común entre los dos dataframe ("employee"). 
# La función requiere que la columna tenga el mismo nombre en los dos df.
df3 = pd.merge(df1, df2)
display(df3)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 2. Join uno a muchos

In [62]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

In [63]:
# Notar que cada supervisor pertenece a UN grupo que puede tener MUCHOS empleados.
# En el join entre empleados y supervisores, los empleados aparecerán una sola vez pero los supervisores
# pueden repetirse.
display(df3)
display(df4)
display(pd.merge(df3, df4))

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


### 3. Join muchos a muchos

In [64]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})


In [65]:
# Notar que cada grupo tiene MUCHOS skills asociados y a su vez pueden pertenecer a él MUCHOS empleados
# Por lo tanto el join entre la tabla de skills y la de empleados es de MUCHOS a MUCHOS.
# Vamos a ver en el resultado que tanto los skills como los empleados pueden repetirse.
display(df1)
display(df5)
df6 = pd.merge(df1, df5)
display(df6)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


## Joins por diferentes columnas

Puede suceder que en nuestros dataframes no haya una única columna con el mismo nombre en ambas tablas para poder hacer el join. 
Para resolver este problema, Pandas implementa los parámetros "on", "right_on" y "left_on" donde podemos especificar por cuáles columnas vamos a unir los datos. 

### 1. Join con "on"

In [66]:
display(df1)
display(df2)

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [67]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 2. Join con "left_on" y "right_on"

In [68]:
df7 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [69]:
display(df1)
display(df7)
pd.merge(df1, df7, left_on="employee", right_on="name")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


### 3. Join por más de una columna

In [70]:
df8 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization'],
                    'tools':  ['calculator','desktop computer','laptop computer','server','desktop computer','board']
                    })

In [71]:
# Ahora podemos ver las herramientas por empleado.

display(df6)
display(df8)
pd.merge(df6,df8,left_on=['group','skills'], right_on = ['group','skills'])

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


,group,skills,tools
0,Accounting,math,calculator
1,Accounting,spreadsheets,desktop computer
2,Engineering,coding,laptop computer
3,Engineering,linux,server
4,HR,spreadsheets,desktop computer
5,HR,organization,board


,employee,group,skills,tools
0,Bob,Accounting,math,calculator
1,Bob,Accounting,spreadsheets,desktop computer
2,Jake,Engineering,coding,laptop computer
3,Lisa,Engineering,coding,laptop computer
4,Jake,Engineering,linux,server
5,Lisa,Engineering,linux,server
6,Sue,HR,spreadsheets,desktop computer
7,Sue,HR,organization,board


In [72]:
# Como los nombres de las columnas son iguales, utilizar sólo "on" es equivalente.
pd.merge(df6,df8,on=['group','skills'])

,employee,group,skills,tools
0,Bob,Accounting,math,calculator
1,Bob,Accounting,spreadsheets,desktop computer
2,Jake,Engineering,coding,laptop computer
3,Lisa,Engineering,coding,laptop computer
4,Jake,Engineering,linux,server
5,Lisa,Engineering,linux,server
6,Sue,HR,spreadsheets,desktop computer
7,Sue,HR,organization,board


## Tipos de joins

### 1. Left joins

In [73]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [74]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [75]:
# Utilizando el valor left en la forma del join produce una lista completa de las filas de df_a, 
# con las filas que matchean de df_b. Si no hay matcheo, las columnas que vienen de df_b serán nulas:
pd.merge(df_a, df_b, on='subject_id', how='left')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


### Check: Cual sería el resultado de cambiar left por right?

In [76]:
pd.merge(df_a, df_b, on='subject_id', how='right')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black
2,6,NaN,NaN,Bran,Balwner
3,7,NaN,NaN,Bryce,Brice
4,8,NaN,NaN,Betty,Btisan


### 2. Inner y outer join

In [77]:
# Como mencionamos antes utilizar la forma outer (OUTER JOIN) 
# produce un conjunto de todas las filas en df_a o df_b. 
# Todas las columnas tendran valores si la fila de un lado tiene su correspondiente en el otro.
# Si no hay matcheo las columnas del que no había valor se completan con null.
pd.merge(df_a, df_b, on='subject_id', how='outer')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black
5,6,NaN,NaN,Bran,Balwner
6,7,NaN,NaN,Bryce,Brice
7,8,NaN,NaN,Betty,Btisan


### Check: Qué pasaría si usamos un inner join?

In [78]:
pd.merge(df_a, df_b, on='subject_id', how='inner')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black


### Encontrando los casos que pertenecen a un dataframe pero no a otro con left join.

Un problema común que podemos querer resolver es encontrar los casos que se encuentran en una tabla pero no en otra. Esto se puede hacer fácilmente con un left join por el o los campos que conforman la clave. 


In [79]:
df1 = pd.DataFrame(data = {'col1' : [1, 2, 3, 4, 5], 'col2' : [10, 11, 12, 13, 14]}) 
df2 = pd.DataFrame(data = {'col1' : [1, 2, 3], 'col2' : [10, 11, 12]})
display(df1)
display(df2)

,col1,col2
0,1,10
1,2,11
2,3,12
3,4,13
4,5,14


,col1,col2
0,1,10
1,2,11
2,3,12


In [80]:
# Agrego en ambos dataframes unas claves que toman siempre el mismo valor para después poder compararlas.
df1['key1'] = 1
df2['key2'] = 1
display(df1)
display(df2)

,col1,col2,key1
0,1,10,1
1,2,11,1
2,3,12,1
3,4,13,1
4,5,14,1


,col1,col2,key2
0,1,10,1
1,2,11,1
2,3,12,1


In [81]:
# Cuando hacemos el left join, los valores de key2 se llenan con null para los valores de df1 que no existen en df2
df1 = pd.merge(df1, df2, on=['col1', 'col2'], how = 'left')
df1

,col1,col2,key1,key2
0,1,10,1,1.0
1,2,11,1,1.0
2,3,12,1,1.0
3,4,13,1,NaN
4,5,14,1,NaN


In [82]:
# Subseteo el resultado del merge para quedarme con los que aparecen en df1 pero no en df2.
df3 = df1[(df1.key2 != df1.key1)]
df3 = df3.drop(['key1','key2'], axis=1)
df3

,col1,col2
3,4,13
4,5,14


### Revisión de buenas prácticas de performance

El método join, cuenta con la misma sintaxis y posibilidades que el método merge pero con la diferencia de que siempre hace la relación por el index. 

In [83]:
# Creamos dos Dataframes de tamaño 1000000
df1 = pd.DataFrame(np.arange(1000000), columns=['A'])
df1['B'] = np.random.randint(1000,size=1000000)
df1.head()

,A,B
0,0,139
1,1,438
2,2,543
3,3,186
4,4,549


In [84]:
df2 = pd.DataFrame(np.arange(1000000), columns=['A2'])
df2['B2'] = np.random.randint(1000,size=1000000)
df2.head()

,A2,B2
0,0,236
1,1,554
2,2,104
3,3,598
4,4,714


In [85]:
%%timeit

# Medimos el tiempo de ejecución del merge:

df1.merge(df2, how='left', left_on='A', right_on='A2')

812 ms ± 85.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%%timeit

# Medimos el tiempo de ejecución del join

df1.set_index('A').join(df2.set_index('A2'), how='left')

284 ms ± 40.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
